Instalação de um conector, para que façamos uma importação

In [2]:
! pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pymysql
import pandas as pd
import warnings
from IPython.display import display, Markdown

In [4]:
def exibir_markdown(texto_md):
    display(Markdown(texto_md))

In [5]:
# DESATIVAR ALERTAS
warnings.filterwarnings('ignore')

In [11]:
drop_exists_cliente_produto = 'DROP TABLE IF EXISTS cliente_produto'
drop_exists_diretor_responsavel = 'DROP TABLE IF EXISTS diretor_responsavel'
drop_exists_categoria = 'DROP TABLE IF EXISTS categoria'
drop_exists_produto = 'DROP TABLE IF EXISTS produto'
drop_exists_tipo_cliente = 'DROP TABLE IF EXISTS tipo_cliente'
drop_exists_cliente = 'DROP TABLE IF EXISTS cliente'


In [12]:
cursor.execute(drop_exists_cliente_produto)
cursor.execute(drop_exists_cliente)
cursor.execute(drop_exists_tipo_cliente)
cursor.execute(drop_exists_produto)
cursor.execute(drop_exists_categoria)
cursor.execute(drop_exists_diretor_responsavel)



0

Sem normalizações 

In [13]:
"""ddl_cliente = ('''
    CREATE TABLE IF NOT EXISTS cliente (
        codigo_cliente INT AUTO_INCREMENT PRIMARY KEY,
        nome_cliente VARCHAR(255) NOT NULL,
        sobrenome_cliente VARCHAR(255) NOT NULL,
        telefones_cliente VARCHAR(255),
        municipio_cliente VARCHAR(255),
        codigo_tipo_cliente INT,
        tipo_cliente VARCHAR(255)
    )
''')"""

"ddl_cliente = ('''\n    CREATE TABLE IF NOT EXISTS cliente (\n        codigo_cliente INT AUTO_INCREMENT PRIMARY KEY,\n        nome_cliente VARCHAR(255) NOT NULL,\n        sobrenome_cliente VARCHAR(255) NOT NULL,\n        telefones_cliente VARCHAR(255),\n        municipio_cliente VARCHAR(255),\n        codigo_tipo_cliente INT,\n        tipo_cliente VARCHAR(255)\n    )\n''')"

In [14]:
"""ddl_produto = ('''
    CREATE TABLE IF NOT EXISTS produto (
        codigo_produto INT AUTO_INCREMENT PRIMARY KEY,
        nome_produto VARCHAR(255) NOT NULL,
        descricao_produto TEXT,
        codigo_tipo_produto INT,
        tipo_produto VARCHAR(255),
        codigo_diretor_responsavel INT,
        nome_diretor_responsavel VARCHAR(255),
        email_diretor_responsavel VARCHAR(255)
    )
''')"""

"ddl_produto = ('''\n    CREATE TABLE IF NOT EXISTS produto (\n        codigo_produto INT AUTO_INCREMENT PRIMARY KEY,\n        nome_produto VARCHAR(255) NOT NULL,\n        descricao_produto TEXT,\n        codigo_tipo_produto INT,\n        tipo_produto VARCHAR(255),\n        codigo_diretor_responsavel INT,\n        nome_diretor_responsavel VARCHAR(255),\n        email_diretor_responsavel VARCHAR(255)\n    )\n''')"